In [1]:
import os

In [2]:
%pwd

'c:\\Users\\midem\\fire-image-classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\midem\\fire-image-classification'

In [5]:
# To be used to update entity (src/fire_image_classifier/entity/config_entity.py)
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
# To be used to update configuration manager (src/fire_image_classifier/config/configuration.py)
from fire_image_classifier.constants import *
from fire_image_classifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
# To be used to update the data ingestion component (src/fire_image_classifier/components/data_ingestion.py
import os
import urllib.request as request
import zipfile
from fire_image_classifier import logger
from fire_image_classifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# To be used to update the data ingestion pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-14 20:33:58,608: INFO: common: yaml file: src\fire_image_classifier\conf\config.yaml loaded successfully]
[2025-01-14 20:33:58,612: INFO: common: yaml file: src\fire_image_classifier\params.yaml loaded successfully]
[2025-01-14 20:33:58,614: INFO: common: created directory at: artifacts]
[2025-01-14 20:33:58,616: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-14 22:26:54,083: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
x-amz-id-2: ECv+dDtXKAbvWi2ZS2iOKwfuV1y7etnxaV8uYuillT/vW4r75jNRgJ6jhVQQToAg6V59VmZadN4=
x-amz-request-id: CAS62ZS22BEC570E
Date: Tue, 14 Jan 2025 20:33:57 GMT
Last-Modified: Wed, 01 Jan 2025 18:35:16 GMT
ETag: "56434de7f92470c065603fcec01f0b23-89"
x-amz-server-side-encryption: AES256
Accept-Ranges: bytes
Content-Type: application/zip
Content-Length: 1522066253
Server: AmazonS3
Connection: close

]
